<a href="https://colab.research.google.com/github/josefrancisco81788/Python-Projects/blob/main/Exploration_Compare_Trading_Strategies_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173913 sha256=296c5ca7869f5b3f1c5c99132ebc312b033060f44e45be2a47e75592bb330bf4
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [9]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf

In [10]:
# Simple Moving Average (SMA) function
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

# MACD function
def MACD(values, n_fast=12, n_slow=26, n_signal=9):
    """
    Calculate MACD and Signal Line
    """
    ema_fast = pd.Series(values).ewm(span=n_fast, min_periods=1).mean()
    ema_slow = pd.Series(values).ewm(span=n_slow, min_periods=1).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=n_signal, min_periods=1).mean()
    return macd, signal

# RSI function
def RSI(values, n=14):
    """
    Calculate Relative Strength Index (RSI)
    """
    delta = pd.Series(values).diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=n).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=n).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Bollinger Bands function
def BollingerBands(values, n=20, n_std=2):
    """
    Calculate Bollinger Bands
    """
    sma = pd.Series(values).rolling(window=n).mean()
    std = pd.Series(values).rolling(window=n).std()
    upper_band = sma + (n_std * std)
    lower_band = sma - (n_std * std)
    return upper_band, lower_band

# Define the SMA crossover strategy class
class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

# Define the MACD strategy class
class MacdStrategy(Strategy):
    n_fast = 12
    n_slow = 26
    n_signal = 9

    def init(self):
        self.macd, self.signal = self.I(MACD, self.data.Close, self.n_fast, self.n_slow, self.n_signal)

    def next(self):
        if crossover(self.macd, self.signal):
            self.position.close()
            self.buy()
        elif crossover(self.signal, self.macd):
            self.position.close()
            self.sell()

# Define the RSI strategy class
class RsiStrategy(Strategy):
    n = 14
    upper = 70
    lower = 30

    def init(self):
        self.rsi = self.I(RSI, self.data.Close, self.n)

    def next(self):
        if self.rsi[-1] < self.lower:
            self.position.close()
            self.buy()
        elif self.rsi[-1] > self.upper:
            self.position.close()
            self.sell()

# Define the Bollinger Bands strategy class
class BollingerBandsStrategy(Strategy):
    n = 20
    n_std = 2

    def init(self):
        self.upper_band, self.lower_band = self.I(BollingerBands, self.data.Close, self.n, self.n_std)

    def next(self):
        if self.data.Close[-1] < self.lower_band[-1]:
            self.position.close()
            self.buy()
        elif self.data.Close[-1] > self.upper_band[-1]:
            self.position.close()
            self.sell()

# Function to find the best strategy
def find_best_strategy(ticker):
    # Download data using yfinance
    data = yf.download(ticker, start="2000-01-01", end="2023-01-01")
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

    # Define the backtest instances for each strategy
    bt_sma = Backtest(data, SmaCross, cash=10_000, commission=.002)
    stats_sma = bt_sma.optimize(
        n1=range(5, 30, 5),
        n2=range(10, 70, 5),
        maximize='Equity Final [$]',
        constraint=lambda param: param.n1 < param.n2
    )

    bt_macd = Backtest(data, MacdStrategy, cash=10_000, commission=.002)
    stats_macd = bt_macd.run()

    bt_rsi = Backtest(data, RsiStrategy, cash=10_000, commission=.002)
    stats_rsi = bt_rsi.run()

    bt_bollinger = Backtest(data, BollingerBandsStrategy, cash=10_000, commission=.002)
    stats_bollinger = bt_bollinger.run()

    # Create a DataFrame for easy comparison
    comparison = pd.DataFrame({
        'Metric': stats_sma.index,
        'SMA Strategy': stats_sma.values,
        'MACD Strategy': stats_macd.values,
        'RSI Strategy': stats_rsi.values,
        'Bollinger Bands Strategy': stats_bollinger.values
    })
    print(comparison)

In [11]:
# Example usage
find_best_strategy('AAPL')

[*********************100%***********************]  1 of 1 completed


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

                    Metric                                       SMA Strategy  \
0                    Start                                2000-01-03 00:00:00   
1                      End                                2022-12-30 00:00:00   
2                 Duration                                 8397 days 00:00:00   
3        Exposure Time [%]                                          98.738552   
4         Equity Final [$]                                      168136.934936   
5          Equity Peak [$]                                      234035.340217   
6               Return [%]                                        1581.369349   
7    Buy & Hold Return [%]                                       12900.253652   
8        Return (Ann.) [%]                                          13.076557   
9    Volatility (Ann.) [%]                                          42.707696   
10            Sharpe Ratio                                           0.306187   
11           Sortino Ratio  